# competition data cleaning

## generate csv files

In [ ]:
import json, os, shutil, re, time
from lxml import etree
from tqdm import tqdm

def get_ajax_json(parameter, page_size):
    # parameter is the parameter of json url.
    # it looks like "&competition=9988"
    import requests
    num = 0
    url = "https://www.kaggle.com/kernels.json?sortBy=hotness&group=everyone{}&pageSize={}".format(parameter,page_size)
    next_url = url
    while True:
        r = requests.get(next_url,
                         auth=('user', 'pass')).json()
        if len(r) == 0:
            break
        elif len(r) > 0 and len(r) < page_size:
            num += len(r)
            break
        else:
            num += len(r)
            last_id = r[len(r) - 1]["id"]
            next_url = url + "&after={}".format(last_id)
    return num

Author_path = r"J:\EECS_6414\Data\19-02-24\competition_split"
competition_file = os.path.join(Author_path, "competition_stats_{}.csv")
log_file = os.path.join(Author_path, "logs.txt")
process_file = os.path.join(Author_path, "process.txt") #time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
header_info = "competition,organization,size,discussion,competitors\n"
START_NUMBERS = [i for i in range(0,2 + 1)]
# dataset: name
# author: name
# competition: yes or not
# size: how many bytes
# discussion: how many discussions if applicable. in "discussion"
# kernel: how many kernels if applicable
# vote: how many votes if applicable
# version: how many versions of dataset if applicable

counter = 0

flag = True
for folder in tqdm(START_NUMBERS):
    if not os.path.isdir(
        os.path.join(Author_path, "{}".format(folder))
    ):
        continue
    c = open(competition_file.format(folder), "w", encoding="utf-8")
    c.write(header_info)  # ",kernels" was the last feature.
    for page_name in tqdm(os.listdir(
        os.path.join(Author_path, "{}".format(folder))
    )):
        try:
            ################################
            ### some kernel pages' name is "kernels_deleted_xxxxxxx".
            ### I don't know why this kind of pages comes into being.
            ### maybe this kernel is just deleted. Simple as it is.
            ### Just ignore it first.
            ################################
            # if page_name != "del=cd0be45190c25545_glove.42b.300d.html" and flag:
            #     continue

            # #### testing:
            # if page_name != "matschiner_nationmaster.html" and flag:
            #     continue
            # flag = False
            ##################################
            author = page_name.split(".")[0]
            with open(
                    os.path.join(
                        Author_path,
                        os.path.join("{}".format(folder), page_name)
                    ), "r", encoding="utf-8"
            ) as f:
                html = f.read()
            page_source = etree.HTML(html)
            ###########
            # this part is for dealing with the problem
            # of: too many request.
            # we have to abondon those "too many requests" pages temprarily.
            ###########

            ############

            print(page_name)
            _ = page_source.xpath("//div[@class='site-layout__main-content']/script[1]/text()")[0]
            ##############################
            # this place occurs because
            # we will fail to crawl such information
            ##############################

            p = re.compile("Kaggle\.State\.push\(\{[\s\S]*\}\)\;")

            s = p.findall(_)[0].replace("Kaggle.State.push(", "")[0:-2]
            info = json.loads(s)
            #####################
            ## some pages will contain kernel page infomation.
            ## this is supposed to be the error of scrapy.
            ## for now, lets' just skip this kind of error.
            #####################

            ##########################
            ##Some pages will still have problem, so let's just
            ## skip them.
            ##########################
            # it is a competition
            c.write("{},{},{},{},{}\n".format(#,{}
                info["competitionSlug"].replace(",","-"), info["organizationSlug"],
                info["databundle"]["totalSize"], info["totalDiscussions"],
                info["totalCompetitors"] #,get_ajax_json("&competitionId={}".format(info["competitionId"]), 200)
                )
            )
        except:
            with open(log_file, "a") as log:
                log.write("{},{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                              folder,
                                              page_name)
                          )
            continue
    with open(process_file, "a") as pro:
        pro.write("{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                folder
                                )
                  )
    c.close()
    print("The {} is хорошо.".format(folder))

## combine csv files

In [ ]:
import json, os, shutil, re, time, csv
from lxml import etree
from tqdm import tqdm

Author_path = r"J:\EECS_6414\Data\19-02-24\competition_split"
competition_file = os.path.join(Author_path, "competition_stats_{}.csv")
combined_file = os.path.join(Author_path, "competition_stats.csv")
START_NUMBERS = [i for i in range(0,2 + 1)]
header_info = "competition,organization,size,discussion,competitors\n"
total_info = []
for folder in tqdm(START_NUMBERS):
    csv_file = competition_file.format(folder)
    if not os.path.exists(csv_file):
        continue
    with open(csv_file, "r", encoding= "utf-8") as f:
        l = list(csv.reader(f))
        total_info.extend(l[1:])
with open(combined_file, "w", encoding="utf-8", newline='') as f:
    f.write(header_info)
    for line in total_info:
        spamwriter = csv.writer(f)
        spamwriter.writerow(line)


# dataset data cleaning

## generate csv files

In [ ]:
import json, os, shutil, re, time
from lxml import etree
from tqdm import tqdm

def get_ajax_json(parameter, page_size):
    # parameter is the parameter of json url.
    # it looks like "&competition=9988"
    import requests
    num = 0
    url = "https://www.kaggle.com/kernels.json?sortBy=hotness&group=everyone{}&pageSize={}".format(parameter,page_size)
    next_url = url
    while True:
        r = requests.get(next_url,
                         auth=('user', 'pass')).json()
        if len(r) == 0:
            break
        elif len(r) > 0 and len(r) < page_size:
            num += len(r)
            break
        else:
            num += len(r)
            last_id = r[len(r) - 1]["id"]
            next_url = url + "&after={}".format(last_id)
    return num

Author_path = r"J:\EECS_6414\Data\19-02-24\dataset_split"
competition_file = os.path.join(Author_path, "dataset_stats_{}.csv")
log_file = os.path.join(Author_path, "logs.txt")
process_file = os.path.join(Author_path, "process.txt") #time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
header_info = "dataset,author,size,discussions,kernels,downloads,views,vote,version\n"
START_NUMBERS = [i for i in range(1,78 + 1)]
# dataset: name
# author: name
# competition: yes or not
# size: how many bytes
# discussion: how many discussions if applicable. in "discussion"
# kernel: how many kernels if applicable
# vote: how many votes if applicable
# version: how many versions of dataset if applicable

counter = 0

flag = True
for folder in tqdm(START_NUMBERS):
    if not os.path.isdir(
        os.path.join(Author_path, "{}".format(folder))
    ):
        continue
    c = open(competition_file.format(folder), "w", encoding="utf-8")
    c.write(header_info)  # ",kernels" was the last feature.
    for page_name in tqdm(os.listdir(
        os.path.join(Author_path, "{}".format(folder))
    )):
        try:
            ################################
            ### some kernel pages' name is "kernels_deleted_xxxxxxx".
            ### I don't know why this kind of pages comes into being.
            ### maybe this kernel is just deleted. Simple as it is.
            ### Just ignore it first.
            ################################
            # if page_name != "del=cd0be45190c25545_glove.42b.300d.html" and flag:
            #     continue

            # #### testing:
            # if page_name != "matschiner_nationmaster.html" and flag:
            #     continue
            # flag = False
            ##################################
            author = page_name.split(".")[0]
            with open(
                    os.path.join(
                        Author_path,
                        os.path.join("{}".format(folder), page_name)
                    ), "r", encoding="utf-8"
            ) as f:
                html = f.read()
            page_source = etree.HTML(html)
            ###########
            # this part is for dealing with the problem
            # of: too many request.
            # we have to abondon those "too many requests" pages temprarily.
            ###########

            ############

            print(page_name)
            _ = page_source.xpath("//div[@class='site-layout__main-content']/script[1]/text()")[0]
            ##############################
            # this place occurs because
            # we will fail to crawl such information
            ##############################

            p = re.compile("Kaggle\.State\.push\(\{[\s\S]*\}\)\;")

            s = p.findall(_)[0].replace("Kaggle.State.push(", "")[0:-2]
            info = json.loads(s)
            #####################
            ## some pages will contain kernel page infomation.
            ## this is supposed to be the error of scrapy.
            ## for now, lets' just skip this kind of error.
            #####################

            ##########################
            ##Some pages will still have problem, so let's just
            ## skip them.
            ##########################
            # it is a competition
            c.write("{},{},{},{},{},{},{},{},{}\n".format(
                info["slug"].replace(",","-"),
                info["owner"]["slug"], info["data"]["totalSize"],
                info["numberOfTopics"], info["numberOfScripts"],
                info["numberOfDownloads"], info["numberOfViews"],
                info["voteCount"], info["datasetVersionTotalCount"]
            )
            )
        except:
            with open(log_file, "a") as log:
                log.write("{},{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                              folder,
                                              page_name)
                          )
            continue
    with open(process_file, "a") as pro:
        pro.write("{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                folder
                                )
                  )
    c.close()
    print("The {} is хорошо.".format(folder))

## combine csv files

In [ ]:
import json, os, shutil, re, time, csv
from lxml import etree
from tqdm import tqdm

Author_path = r"J:\EECS_6414\Data\19-02-24\dataset_split"
competition_file = os.path.join(Author_path, "dataset_stats_{}.csv")
combined_file = os.path.join(Author_path, "dataset_stats.csv")
START_NUMBERS = [i for i in range(0,78 + 1)]
header_info = "dataset,author,size,discussions,kernels,downloads,views,vote,version\n"
total_info = []
for folder in tqdm(START_NUMBERS):
    csv_file = competition_file.format(folder)
    if not os.path.exists(csv_file):
        continue
    with open(csv_file, "r", encoding= "utf-8") as f:
        l = list(csv.reader(f))
        total_info.extend(l[1:])
with open(combined_file, "w", encoding="utf-8", newline='') as f:
    f.write(header_info)
    for line in total_info:
        spamwriter = csv.writer(f)
        spamwriter.writerow(line)

# Kernel data cleaning

## generate csv files

In [ ]:
import json, os, shutil, re, time
from lxml import etree
from os.path import join
import tqdm
import pandas as pd
from statistics import mean

def one_shot(stats, sourceType, list_of_feature, names):
    # renew the list of list
    # and return invalid_flag
    list_of_list = []
    for i in range(len(list_of_feature)):
        list_of_list.append([])
    for name in names:
        cmp_info = stats[stats[sourceType] == name]
        if len(cmp_info) == 0:
            print("{} this competition has no information".format(name))
            return False, list_of_list
        for l, f in zip(list_of_list, list_of_feature):
            l.extend(cmp_info[f].tolist())
    return True, list_of_list

competition_stats = pd.read_csv(r"J:\EECS_6414\Data\19-02-24\competition_split/competition_stats.csv")#,encoding ='latin1'
competition_stats.loc[-1] = ["None", "None", 0, 0, 0]  # adding a row
competition_stats.reset_index(drop= True, inplace = True)

dataset_stats = pd.read_csv(r"J:\EECS_6414\Data\19-02-24\dataset_split/dataset_stats.csv",encoding ='utf-8')
dataset_stats.loc[-1] = ["None", "None", 0, 0, 0, 0, 0, 0, 0]  # adding a row
dataset_stats.reset_index(drop= True, inplace = True)

TOP_DIR = r"J:\EECS_6414\Data\19-02-24\kernel_split"
log_file = os.path.join(TOP_DIR, "logs.txt")
process_file = os.path.join(TOP_DIR, "process.txt") #time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),

for num in tqdm.tqdm(range(2, 1049 + 1)):
    file_folder = join(TOP_DIR, "{}".format(num))
    if not os.path.exists(file_folder):
        continue

    kernel_stats_files = r"J:\EECS_6414\Data\19-02-24\kernel_split\cleaning_result\kernel_stats_combined_{}.csv".format(num)
    k = open(kernel_stats_files, "w", encoding="utf-8")
    k.write("kernel,author,dataSources,versions,forks,comments,views,votes,"  # 8
            "competitions,"  # 1 + 3 * 4 = 13
            "competition_size_ttl,competition_size_avg,competition_size_max,competition_size_min,"
            "competition_discussion_ttl,competition_discussion_avg,competition_discussion_max,competition_discussion_min,"
            "competition_competitor_ttl,competition_competitor_avg,competition_competitor_max,competition_competitor_min,"
            "datasets,"  # 1 + 7 * 4 = 29
            "dataset_size_ttl,dataset_size_avg,dataset_size_max,dataset_size_min,"
            "dataset_discussion_ttl,dataset_discussion_avg,dataset_discussion_max,dataset_discussion_min,"
            "dataset_kernel_ttl,dataset_kernel_avg,dataset_kernel_max,dataset_kernel_min,"
            "dataset_download_ttl,dataset_download_avg,dataset_download_max,dataset_download_min,"
            "dataset_view_ttl,dataset_view_avg,dataset_view_max,dataset_view_min,"
            "dataset_vote_ttl,dataset_vote_avg,dataset_vote_max,dataset_vote_min,"
            "dataset_version_ttl,dataset_version_avg,dataset_version_max,dataset_version_min"
            "\n")
    files = os.listdir(file_folder)
    for file in tqdm.tqdm(files):
        try:
            with open(os.path.join(file_folder, file), "r", encoding="utf-8") as f:
                html = f.read()
            page_source = etree.HTML(html)
            ##################
            # sometimes, the html parsing can get none.
            # just skip this kind of situation
            ##################
            if page_source == None:
                continue
            ###########
            # this part is for dealing with the problem
            # of: too many request.
            # we have to abondon those "too many requests" pages temprarily.
            ###########
            if page_source.xpath("/html/body/pre/text()") != []:
                print("bad file: {}".format("{}/{}".format(num, file)))
                continue
            ############
            _ = page_source.xpath("//div[@class='site-layout__main-content']/script[1]/text()")[0]
            p = re.compile("Kaggle\.State\.push\(\{[\s\S]*\}\)\;")

            s = p.findall(_)[0].replace("Kaggle.State.push(", "")[0:-2]
            info = json.loads(s)

            ## about versions and comments
            versions = 0
            comments = 0
            for menulink in info["menuLinks"]:
                if menulink["title"] == "Versions":
                    versions = menulink["count"]
                elif menulink["title"] == "Comments":
                    comments = menulink["count"]
                else:
                    pass
            ####about dataset

            source = None
            sourceType = None
            tmp = info["dataSources"]
            dataSources = len(tmp)
            if tmp != []:
                source = tmp[0]["slug"].replace(",", " ")
                sourceType = tmp[0]["sourceType"]
            ##############
            # divide the datasets: competition and dataset
            ##############
            cpt = []
            dts = []
            cpt_num = 0
            dts_num = 0
            for t in tmp:
                if t["sourceType"] == "competition":
                    cpt.append(t["slug"])
                    cpt_num += 1
                elif t["sourceType"] == "dataset":
                    dts.append(t["slug"])
                    dts_num += 1
            if cpt == []:
                cpt.append("None")
            if dts == []:
                dts.append("None")

            valid_flag, cpt_rst = one_shot(competition_stats,
                                           "competition",
                                          ["size", "discussion", "competitors"],
                                          cpt)
            if not valid_flag:
                continue

            valid_flag, dts_rst = one_shot(dataset_stats,
                                           "dataset",
                                          ["size", "discussions", "kernels", "downloads", "views", "vote", "version"],
                                          dts)
            if not valid_flag:
                continue
            # saving to file
            k.write((",".join(["{}"] * 50) + "\n").format(
                info["kernel"]["slug"], info["kernel"]["author"]["userName"], dataSources,
                versions, info["kernel"]["forkCount"], comments, info["kernel"]["viewCount"], info["kernel"]["upvoteCount"],
                ### 42 features to add
                ### sum, avg, max, min
                cpt_num,
                sum(cpt_rst[0]), mean(cpt_rst[0]), max(cpt_rst[0]), min(cpt_rst[0]),
                sum(cpt_rst[1]), mean(cpt_rst[1]), max(cpt_rst[1]), min(cpt_rst[1]),
                sum(cpt_rst[2]), mean(cpt_rst[2]), max(cpt_rst[2]), min(cpt_rst[2]),

                dts_num,
                sum(dts_rst[0]), mean(dts_rst[0]), max(dts_rst[0]), min(dts_rst[0]),
                sum(dts_rst[1]), mean(dts_rst[1]), max(dts_rst[1]), min(dts_rst[1]),
                sum(dts_rst[2]), mean(dts_rst[2]), max(dts_rst[2]), min(dts_rst[2]),
                sum(dts_rst[3]), mean(dts_rst[3]), max(dts_rst[3]), min(dts_rst[3]),
                sum(dts_rst[4]), mean(dts_rst[4]), max(dts_rst[4]), min(dts_rst[4]),
                sum(dts_rst[5]), mean(dts_rst[5]), max(dts_rst[5]), min(dts_rst[5]),
                sum(dts_rst[6]), mean(dts_rst[6]), max(dts_rst[6]), min(dts_rst[6]),
                )
            )
        except:
            with open(log_file, "a") as log:
                log.write("{},{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                              num,
                                              file)
                          )
            continue

    with open(process_file, "a") as pro:
        pro.write("{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                   num
                                   )
                  )
    k.close()
    print("The kernel folder {} is хорошо..".format(num))

In [3]:
import json, os, shutil, re, time, csv
from lxml import etree
from tqdm import tqdm

Author_path = r"J:\EECS_6414\Data\19-02-24\kernel_split\cleaning_result\measure_set_0"
competition_file = os.path.join(Author_path, "kernel_stats_combined_{}.csv")
combined_file = os.path.join(Author_path, "kernel_stats_combined.csv")
START_NUMBERS = [i for i in range(0,1049 + 1)]
header_info = ("kernel,author,dataSources,versions,forks,comments,views,votes,"  # 8
            "competitions,"  # 1 + 3 * 4 = 13
            "competition_size_ttl,competition_size_avg,competition_size_max,competition_size_min,"
            "competition_discussion_ttl,competition_discussion_avg,competition_discussion_max,competition_discussion_min,"
            "competition_competitor_ttl,competition_competitor_avg,competition_competitor_max,competition_competitor_min,"
            "datasets,"  # 1 + 7 * 4 = 29
            "dataset_size_ttl,dataset_size_avg,dataset_size_max,dataset_size_min,"
            "dataset_discussion_ttl,dataset_discussion_avg,dataset_discussion_max,dataset_discussion_min,"
            "dataset_kernel_ttl,dataset_kernel_avg,dataset_kernel_max,dataset_kernel_min,"
            "dataset_download_ttl,dataset_download_avg,dataset_download_max,dataset_download_min,"
            "dataset_view_ttl,dataset_view_avg,dataset_view_max,dataset_view_min,"
            "dataset_vote_ttl,dataset_vote_avg,dataset_vote_max,dataset_vote_min,"
            "dataset_version_ttl,dataset_version_avg,dataset_version_max,dataset_version_min"
            "\n")
total_info = []
for folder in tqdm(START_NUMBERS):
    csv_file = competition_file.format(folder)
    if not os.path.exists(csv_file):
        continue
    with open(csv_file, "r", encoding= "utf-8") as f:
        l = list(csv.reader(f))
        total_info.extend(l[1:])
with open(combined_file, "w", encoding="utf-8", newline='') as f:
    f.write(header_info)
    for line in total_info:
        spamwriter = csv.writer(f)
        spamwriter.writerow(line)

100%|██████████████████████████████████████████████████████████████████████████████| 1050/1050 [00:17<00:00, 60.20it/s]


## new features for kernel

In [ ]:
import os, json, tqdm, time
from os.path import join

kernelInfo_dir = "J:\\EECS_6414\\Data\\19-02-19"

cleaning_result_path = r"J:\EECS_6414\Data\19-02-24\kernel_split\cleaning_result\measure_set_1"
log_file = os.path.join(cleaning_result_path, "logs.txt")
cleaning_stats = os.path.join(cleaning_result_path, "kernel_stats_{}.csv")
if not os.path.exists(cleaning_result_path):
    os.makedirs(cleaning_result_path)

header_info = "kernel,author,Hidden,Notebook,null,Data,Visualization,Other,language,isNotebook,tags\n"

for num in tqdm.tqdm(range(0, 500)):
    kernelInfo_file = join(kernelInfo_dir, "KernelInfo_{}.json".format(num))
    if not os.path.exists(kernelInfo_file):
        continue

    cleaning_stats_file = open(cleaning_stats.format(num), "w")
    cleaning_stats_file.write(header_info)

    with open(kernelInfo_file, "r", encoding="utf-8") as f:
        l = json.load(f)
    for kernel in l:
        try:
            info = kernel["info"]

            k_id = info["scriptUrl"].split("/")[-1]
            a_id = info["author"]["userName"]

            outputTypes = {'Hidden': 0, 'Notebook': 0, 'null': 0, 'Data': 0, 'Visualization': 0, 'Other': 0}
            for outputtype in info["outputTypes"]:
                _ = outputtype["extension"]
                if _ == None:
                    outputTypes["null"] = outputtype["count"]
                    continue
                outputTypes[_] = outputtype["count"]
            outputTypes_str = ",".join(["{}".format(s) for s in list(outputTypes.values())])

            language = info["languageName"]

            isNotebook = True if (language == "R" and info["aceLanguageName"] == "markdown" and info[
                "isNotebook"] == "False") else info["isNotebook"]

            tags_list = []
            tags = ""
            for category in info["categories"]:
                tags_list.append(category["name"].replace(" ", "-"))
            tags = ";".join(tags_list)

            cleaning_stats_file.write("{},{},{},{},{},{}\n".format(
                k_id, a_id,
                outputTypes_str,
                language,
                isNotebook,
                tags
            ))

        except:
            with open(log_file, "a") as ff:
                ff.write("{},{},{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                             num, k_id, a_id))
            continue
    cleaning_stats_file.close()

print("хорошо.")

In [5]:
import json, os, shutil, re, time, csv
from lxml import etree
from tqdm import tqdm

Author_path = r"J:\EECS_6414\Data\19-02-24\kernel_split\cleaning_result\measure_set_1"
competition_file = os.path.join(Author_path, "kernel_stats_{}.csv")
combined_file = os.path.join(Author_path, "kernel_stats.csv")
START_NUMBERS = [i for i in range(0,500 + 1)]
header_info = ("kernel,author,Hidden,Notebook,null,Data,Visualization,Other,language,isNotebook,tags\n")
total_info = []
for folder in tqdm(START_NUMBERS):
    csv_file = competition_file.format(folder)
    if not os.path.exists(csv_file):
        continue
    with open(csv_file, "r", encoding= "utf-8") as f:
        l = list(csv.reader(f))
        total_info.extend(l[1:])
with open(combined_file, "w", encoding="utf-8", newline='') as f:
    f.write(header_info)
    for line in total_info:
        spamwriter = csv.writer(f)
        spamwriter.writerow(line)

100%|███████████████████████████████████████████████████████████████████████████████| 501/501 [00:00<00:00, 849.62it/s]


# Code cleaning

## see if there is some special kind of suffix in code files. 

In [ ]:
##jupyter nbconvert --to script [YOUR_NOTEBOOK].ipynb
import json, os, shutil, re, time
from lxml import etree
from tqdm import tqdm

origin_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code"
script_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\python_script"

suffixes = {}

START_NUMBERS = [i for i in range(0, 228 + 1)]
for folder in tqdm(START_NUMBERS):
    if not os.path.isdir(
            os.path.join(origin_path, "{}".format(folder))
    ):
        continue

    for page_name in tqdm(os.listdir(
            os.path.join(origin_path, "{}".format(folder))
    )):
        suffix = page_name.split(".")[-1]
        suffixes[suffix] = 1

In [2]:
print(suffixes)

{'ipynb': 1, 'py': 1, 'irnb': 1, 'Rmd': 1, 'R': 1, 'xpynb': 1, 'sql': 1, 'xrnb': 1, 'jl': 1, 'ijlnb': 1}


## change the .ipynb file to python script. For now just focus on python files. 

名剑惹风流，神威荡九州。

In [ ]:
##jupyter nbconvert --to script [YOUR_NOTEBOOK].ipynb
import json, os, shutil, re, time
from lxml import etree
from tqdm import tqdm

origin_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code"
script_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\python_script"
log_file = os.path.join(script_path, "log.csv")

START_NUMBERS = [i for i in range(228, 228 + 1)]
for folder in tqdm(START_NUMBERS):
    if not os.path.isdir(
            os.path.join(origin_path, "{}".format(folder))
    ):
        continue

    origin_file_path = os.path.join(origin_path, "{}".format(folder))
    new_file_path = os.path.join(script_path, "{}".format(folder))
    if not os.path.exists(new_file_path):
        os.makedirs(new_file_path)

    for page_name in tqdm(os.listdir(
            os.path.join(origin_path, "{}".format(folder))
    )):

        if ".ipynb" not in page_name:
            continue
        try:

            origin_file = os.path.join(origin_file_path, page_name)

            new_name = page_name.replace("ipynb", "py")

            intermediate_file = os.path.join(origin_file_path, new_name)

            new_file = os.path.join(new_file_path, new_name)

            os.system("jupyter nbconvert {} --to python".format(origin_file))
            shutil.move(intermediate_file, new_file)
        except:
            with open(log_file, "a") as f:
                f.write("{},{}\n".format(folder, page_name))

### mind the "xpynb" type of files. change the "xpynb" files to python script

In [ ]:
##jupyter nbconvert --to script [YOUR_NOTEBOOK].ipynb
import json, os, shutil, re, time
from lxml import etree
from tqdm import tqdm

origin_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code"
script_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\python_script"
log_file = os.path.join(script_path, "log.csv")

START_NUMBERS = [i for i in range(228, 228 + 1)]
for folder in tqdm(START_NUMBERS):
    if not os.path.isdir(
            os.path.join(origin_path, "{}".format(folder))
    ):
        continue

    origin_file_path = os.path.join(origin_path, "{}".format(folder))
    new_file_path = os.path.join(script_path, "{}".format(folder))
    if not os.path.exists(new_file_path):
        os.makedirs(new_file_path)

    for page_name_x in tqdm(os.listdir(origin_file_path)):

        if ".xpynb" not in page_name_x:
            continue
        try:
            
            page_name = page_name_x.replace("xpynb", "ipynb")
            shutil.copy(
                os.path.join(origin_file_path, page_name_x), 
                os.path.join(origin_file_path, page_name)
            )
            
            origin_file = os.path.join(origin_file_path, page_name)
            

            new_name = page_name.replace("ipynb", "py")

            intermediate_file = os.path.join(origin_file_path, new_name)

            new_file = os.path.join(new_file_path, new_name)

            os.system("jupyter nbconvert {} --to python".format(origin_file))
            shutil.move(intermediate_file, new_file)
            os.remove(os.path.join(origin_file_path, page_name))
        except:
            with open(log_file, "a") as f:
                f.write("{},{}\n".format(folder, page_name_x))

## move the original .py files to python_script
not all of the codes are in .ipynb, some are in .py originally. So lets' consider this type of code as well. 

In [ ]:
##jupyter nbconvert --to script [YOUR_NOTEBOOK].ipynb
import json, os, shutil, re, time
from lxml import etree
from tqdm import tqdm

origin_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code"
script_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\python_script"
log_file = os.path.join(script_path, "log_origin_py.csv")

START_NUMBERS = [i for i in range(228, 228 + 1)]
for folder in tqdm(START_NUMBERS):
    if not os.path.isdir(
            os.path.join(origin_path, "{}".format(folder))
    ):
        continue

    origin_file_path = os.path.join(origin_path, "{}".format(folder))
    new_file_path = os.path.join(script_path, "{}".format(folder))
    if not os.path.exists(new_file_path):
        os.makedirs(new_file_path)

    for page_name in tqdm(os.listdir(
            os.path.join(origin_path, "{}".format(folder))
    )):
        try:
#             if page_name.split(".")[-1] != ".py": 
#                 continue
            if ".py" in page_name:
                print(page_name)
                origin_file = os.path.join(origin_file_path, page_name)
                new_file = os.path.join(new_file_path, page_name)
                shutil.copy(
                    origin_file, 
                    new_file
                )
        except:
            with open(log_file, "a") as f:
                f.write("{},{}\n".format(folder, page_name))

## parse the .py files, get some metrics

In [ ]:
import json, os, shutil, re, time
from lxml import etree
from tqdm import tqdm
import pygount, ast
from _ast import *

def parse_code(file_path):
    analysis = pygount.source_analysis(file_path, "???")
    sloc = analysis[3] + 1 ### this is rough, because this is only the bug I can see now.
    comment_rough = analysis[4]
    empty = analysis[5] - 1

    with open(file_path, "r") as f:
        source = "".join(f.readlines())
    AST = ast.parse(source, '<unknow>', 'exec')
    w = ast.walk(AST)
    line_no = {}
    import_list = []
    import_from_list = []
    control_count = 0 # control means: (condition)If, (loop)For, While, (others)Continue, Pass, Break
    for i in w:
        if isinstance(i, Import):
            import_list.extend([alia.name for alia in i.names])

        elif isinstance(i, ImportFrom):
            import_from_list.extend([i.module + ".{}".format(alia.name) for alia in i.names])
        elif isinstance(i, If) or \
            isinstance(i, For) or \
            isinstance(i, While) or \
            isinstance(i, Continue) or \
            isinstance(i, Pass) or \
            isinstance(i, Break):
            control_count += 1
        else:
            pass
    mccabe = control_count - 2
    import_from_list.sort()
    import_list.sort()

    return [sloc, comment_rough, empty, mccabe, ";".join(import_list), ";".join(import_from_list)]

Author_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\python_script"

cleaning_result_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\python_script_clean_result\measures_from_pyscript"
competition_file = os.path.join(cleaning_result_path, "code_stats_pyscript_{}.csv")
# cleaning_result_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\python_script_clean_result\measures_from_origin"
# competition_file = os.path.join(cleaning_result_path, "code_stats_origin_{}.csv")
if not os.path.exists(cleaning_result_path):
    os.makedirs(cleaning_result_path)

log_file = os.path.join(cleaning_result_path, "logs.txt")
process_file = os.path.join(cleaning_result_path, "process.txt") #time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
header_info = "kernel,author,sloc,comments,empty,mccabe,import,import_from\n"
START_NUMBERS = [i for i in range(0,300 + 1)]

counter = 0

flag = True
for folder in tqdm(START_NUMBERS):
    if not os.path.isdir(
        os.path.join(Author_path, "{}".format(folder))
    ):
        continue
    c = open(competition_file.format(folder), "w", encoding="utf-8")
    c.write(header_info)  # ",kernels" was the last feature.
    files_path = os.path.join(Author_path, "{}".format(folder))
    for page_name in (os.listdir(files_path)):
        try:
            parse_result = parse_code(
                os.path.join(files_path, page_name)
            )
            c.write("{},{},".format(page_name.split("_")[-1].split(".")[0], page_name.split("_")[-2]) +
                    ",".join(["{}".format(item) for item in parse_result]) +
                    "\n")
        except:
            with open(log_file, "a") as log:
                log.write("{},{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                              folder,
                                              page_name)
                          )
            continue
    with open(process_file, "a") as pro:
        pro.write("{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                folder
                                )
                  )
    c.close()
    print("The {} is хорошо.".format(folder))

## merge the code measures

In [ ]:
import json, os, shutil, re, time, csv
from lxml import etree
from tqdm import tqdm

Author_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code_clean_result\measures_from_origin"
competition_file = os.path.join(Author_path, "code_stats_origin_{}.csv")
combined_file = os.path.join(Author_path, "code_stats_origin.csv")
START_NUMBERS = [i for i in range(0, 230 + 1)]
header_info = ("kernel,author,sloc,comments,empty,mccabe,import,import_from\n")
total_info = []
for folder in tqdm(START_NUMBERS):
    csv_file = competition_file.format(folder)
    if not os.path.exists(csv_file):
        continue
    with open(csv_file, "r", encoding= "utf-8") as f:
        l = list(csv.reader(f))
        total_info.extend(l[1:])
with open(combined_file, "w", encoding="utf-8", newline='') as f:
    f.write(header_info)
    for line in total_info:
        spamwriter = csv.writer(f)
        spamwriter.writerow(line)

## parse the original code files, especially for python notebook .ipynb
### and we will also parse .py files. But those return values of .py files will all be 0. 

In [ ]:
import json, os, shutil, re, time
from lxml import etree
from tqdm import tqdm
import pygount, ast
from _ast import *


def parse_notebook(path):
    num_of_cell = 0
    num_of_output = 0
    ####据统计，就只有这么四种输出了。
    output_type_count = {"display_data": 0, "stream": 0, "execute_result": 0, "error": 0}  #
    try:
        #####################
        ## why try here?
        ## because if it is not a notebook, it will not open by json.load, and it will return error.
        ## retunr error, we will just simply return multiple 0s.
        #####################
        with open(path, "r", encoding="utf-8") as f:
            j = json.load(f)
        for cell in j["cells"]:
            num_of_cell += 1
            if "outputs" not in cell:
                continue
            for output in cell["outputs"]:
                num_of_output += 1
                output_type = output["output_type"]
                if output_type in output_type_count:
                    output_type_count[output_type] += 1
    except:
        pass

    new_list = [num_of_cell, num_of_output]
    new_list.extend(list(output_type_count.values()))
    return new_list


Author_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code"

cleaning_result_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code_clean_result\measures_from_origin"
competition_file = os.path.join(cleaning_result_path, "code_stats_origin_{}.csv")
if not os.path.exists(cleaning_result_path):
    os.makedirs(cleaning_result_path)

log_file = os.path.join(cleaning_result_path, "logs.txt")
process_file = os.path.join(cleaning_result_path,
                            "process.txt")  # time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
header_info = "kernel,author,num_of_cell,num_of_output,display_data,stream,execute_result,error\n"
START_NUMBERS = [i for i in range(0, 300 + 1)]

counter = 0

flag = True
for folder in tqdm(START_NUMBERS):
    if not os.path.isdir(
            os.path.join(Author_path, "{}".format(folder))
    ):
        continue
    c = open(competition_file.format(folder), "w", encoding="utf-8")
    c.write(header_info)  # ",kernels" was the last feature.
    files_path = os.path.join(Author_path, "{}".format(folder))
    for page_name in (os.listdir(files_path)):
        try:
            suffix = page_name.split(".")[-1]
            if "py" not in suffix:
                continue
            #########################
            #             if (".ipynb" in page_name or ".xpynb" in page_name):
            #                 # something related to python notebook

            #                 pass
            #             elif ".py" in page_name:
            #                 # something related to py script

            #                 pass
            parse_result = parse_notebook(
                os.path.join(files_path, page_name)
            )
            c.write("{},{},".format(page_name.split("_")[-1].split(".")[0], page_name.split("_")[-2]) +
                    ",".join(["{}".format(item) for item in parse_result]) +
                    "\n")
        except:
            with open(log_file, "a") as log:
                log.write("{},{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                              folder,
                                              page_name)
                          )
            continue
    with open(process_file, "a") as pro:
        pro.write("{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                   folder
                                   )
                  )
    c.close()
    print("The {} is хорошо.".format(folder))

## merge the code measures

In [3]:
import json, os, shutil, re, time, csv
from lxml import etree
from tqdm import tqdm

Author_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code_clean_result\measures_from_origin"
competition_file = os.path.join(Author_path, "code_stats_origin_{}.csv")
combined_file = os.path.join(Author_path, "code_stats_origin.csv")
START_NUMBERS = [i for i in range(0, 230 + 1)]
header_info = ("kernel,author,num_of_cell,num_of_output,display_data,stream,execute_result,error\n")
total_info = []
for folder in tqdm(START_NUMBERS):
    csv_file = competition_file.format(folder)
    if not os.path.exists(csv_file):
        continue
    with open(csv_file, "r", encoding= "utf-8") as f:
        l = list(csv.reader(f))
        total_info.extend(l[1:])
with open(combined_file, "w", encoding="utf-8", newline='') as f:
    f.write(header_info)
    for line in total_info:
        spamwriter = csv.writer(f)
        spamwriter.writerow(line)

100%|██████████████████████████████████████████████████████████████████████████████| 231/231 [00:00<00:00, 1172.00it/s]


# merge data tables

## for now we only has the voted kernels' results

In [1]:
import pandas as pd
import os

code_voted_original = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code_clean_result\measures_from_origin\code_stats_origin.csv"
code_voted_pyscript = r"J:\EECS_6414\Data\19-02-24\code_split\voted\python_script_clean_result\measures_from_pyscript\code_stats_pyscript.csv"
competition = r"J:\EECS_6414\Data\19-02-24\competition_split\competition_stats.csv"
datasets = r"J:\EECS_6414\Data\19-02-24\dataset_split\dataset_stats.csv"
kernel_measure_0 = r"J:\EECS_6414\Data\19-02-24\kernel_split\cleaning_result\measure_set_0\kernel_stats_combined.csv"
kernel_measure_1 = r"J:\EECS_6414\Data\19-02-24\kernel_split\cleaning_result\measure_set_1\kernel_stats.csv"

def change_column_names(table, table_identifier, exclude_columns = []):
    origin_col_name = list(table.columns.values)
    new_columns = {old_column: "{}_{}".format(table_identifier, old_column) for old_column in origin_col_name if old_column not in exclude_columns}
    table.rename(index=str, 
                    columns=new_columns, 
                    inplace= True)
    
def change_id_format(table):
    table["id"] = table["kernel"] + table["author"]
    table.drop(['kernel', 'author'], axis=1, inplace= True)
    
cvo = pd.read_csv(code_voted_original, encoding="utf-8")
cvp = pd.read_csv(code_voted_pyscript, encoding="utf-8")
#cpt = pd.read_csv(competition, encoding="utf-8")
#dts = pd.read_csv(datasets, encoding="utf-8")
km0 = pd.read_csv(kernel_measure_0, encoding="utf-8")
km1 = pd.read_csv(kernel_measure_1, encoding="utf-8")
change_column_names(cvo, "cvo", ["kernel", "author"])
change_column_names(cvp, "cvp", ["kernel", "author"])
#change_column_names(cpt, "cpt", ["kernel", "author"])
#change_column_names(dts, "dts", ["kernel", "author"])
change_column_names(km0, "km", ["kernel", "author"])
change_column_names(km1, "km", ["kernel", "author"])
change_id_format(cvo)
change_id_format(cvp)
change_id_format(km0)
change_id_format(km1)
a = pd.merge(cvo, cvp, on= "id")
b = pd.merge(a, km0, on= "id")
c = pd.merge(b, km1, on= "id")
c.to_csv("kernel_datssource_code.csv", index=None)

C:\Users\xmk233\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## calculating the target encoding for multiple tags

### calculate the high and low vote

In [1]:
import pandas as pd
import os

# cvo: code_voted_original
# cvp: code_voted_pyscript
# km: kernel_measures
merged_table = r"J:\EECS6414\process\Pipeline\Data_cleaning\kernel_datssource_code.csv"
mt = pd.read_csv(merged_table, encoding="utf-8")

print(mt["km_votes"].describe())

mt["high_voted"] = None
mt["high_voted"][mt["km_votes"] > 2] = True
mt["high_voted"][mt["km_votes"] <= 2] = False
mt["high_voted"].describe()

mt.to_csv("kernel_datssource_code_-1.csv", index=None)

count    28205.000000
mean        11.483425
std         60.795352
min          0.000000
25%          1.000000
50%          2.000000
75%          7.000000
max       3713.000000
Name: km_votes, dtype: float64


C:\Users\xmk233\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\xmk233\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### target encoding and some new features for tags

In [1]:
import pandas as pd
import os

# cvo: code_voted_original
# cvp: code_voted_pyscript
# km: kernel_measures
merged_table = r"J:\EECS6414\process\Pipeline\Data_cleaning\kernel_datssource_code_-1.csv"
mt = pd.read_csv(merged_table, encoding="utf-8")

kernel_tags = {}
tags_kernel = {}
tags_high_and_low_vote_kernel_number = {}

how_many_kernels_dont_have_tags = 0
how_many_kernels_have_tags = 0

for index, row in mt.iterrows():
    kernel_id = row["id"]
    tag_string = row["km_tags"]
    if not isinstance(tag_string, str):
        how_many_kernels_dont_have_tags += 1
        tags_of_this_kernel = ["<(no_tags)>"]
    else:
        how_many_kernels_have_tags += 1
        tags_of_this_kernel = tag_string.split(";")
    for tag in tags_of_this_kernel:
        #####
        if tag in tags_kernel:
            tags_kernel[tag].append(kernel_id)
        else:
            tags_kernel[tag] = [kernel_id]
        #####
        if tag in tags_high_and_low_vote_kernel_number:
            if row["high_voted"]:
                tags_high_and_low_vote_kernel_number[tag][0] += 1
            else:
                tags_high_and_low_vote_kernel_number[tag][1] += 1
        else:
            tags_high_and_low_vote_kernel_number[tag] = [1, 0] if row["high_voted"] else [0, 1]
    kernel_tags[kernel_id] = tags_of_this_kernel

print(
    "how many kernels have tags: ", how_many_kernels_have_tags, "\n",
    "how_many_kernels_dont_have_tags: ", how_many_kernels_dont_have_tags, "\n",
    # "how many tags are in total: ", len(list(tags_num.keys())), "\n"
    tags_high_and_low_vote_kernel_number
)

def counting_tags(string):
    if isinstance(string, float):
        return 0
    return len(string.split(";"))

def target_encoding_tags_sum(string, return_stats="sum"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_tags)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "sum":
        return sum(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_avg(string, return_stats="avg"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_tags)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if (return_stats == "mean" or return_stats == "avg"):
        return mean(encoding_vals)
    else:
        return 0

def target_encoding_tags_max(string, return_stats="max"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_tags)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "max":
        return max(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_min(string, return_stats="min"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_tags)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "min":
        return min(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_median(string, return_stats="median"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_tags)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "median":
        return median(encoding_vals)
    else:
        return 0

mt["km_num_of_tags"] = 0
mt["km_tags_te_sum"] = None
mt["km_tags_te_avg"] = None
mt["km_tags_te_max"] = None
mt["km_tags_te_min"] = None
mt["km_tags_te_median"] = None

mt["km_num_of_tags"] = mt["km_tags"].map(counting_tags)
mt["km_tags_te_sum"] = mt["km_tags"].map(target_encoding_tags_sum)
mt["km_tags_te_avg"] = mt["km_tags"].map(target_encoding_tags_avg)
mt["km_tags_te_max"] = mt["km_tags"].map(target_encoding_tags_max)
mt["km_tags_te_min"] = mt["km_tags"].map(target_encoding_tags_min)
mt["km_tags_te_median"] = mt["km_tags"].map(target_encoding_tags_median)
# mt["km_target_encoding"] = 0
# mt["km_target_encoding"] = mt["km_tags"].map(target_encoding_tags)
mt.to_csv("kernel_datssource_code_0.csv", index=None)

how many kernels have tags:  6075 
 how_many_kernels_dont_have_tags:  22130 
 {'tutorial': [842, 310], 'feature-engineering': [588, 199], 'healthcare': [82, 13], 'health': [5, 1], 'eda': [1070, 272], 'data-visualization': [1703, 617], 'classification': [704, 364], 'beginner': [1649, 795], 'starter-code': [524, 259], '<(no_tags)>': [9542, 12588], 'intermediate': [25, 3], 'nlp': [180, 54], 'xgboost': [107, 41], 'linear-regression': [81, 47], 'regression': [85, 38], 'time-series-analysis': [31, 11], 'model-explainability': [13, 0], 'dimensionality-reduction': [37, 7], 'pca': [53, 11], 'ensembling': [86, 23], 'model-comparison': [55, 23], 'survey-analysis': [47, 5], 'animation': [11, 3], 'geography': [13, 0], 'terrorism': [10, 3], 'learn': [47, 4], 'gradient-boosting': [94, 29], 'deep-learning': [420, 178], 'finance': [128, 21], 'preprocessing': [92, 28], 'data-cleaning': [411, 196], 'clinical-research': [2, 0], 'cnn': [170, 66], 'multiclass-classification': [62, 24], 'machine-learning': [

### target encoding and some new features for import

In [1]:
import pandas as pd
import os

# cvo: code_voted_original
# cvp: code_voted_pyscript
# km: kernel_measures
merged_table = r"J:\EECS6414\process\Pipeline\Data_cleaning\kernel_datssource_code_0.csv"
mt = pd.read_csv(merged_table, encoding="utf-8")

kernel_tags = {}
tags_kernel = {}
tags_high_and_low_vote_kernel_number = {}

how_many_kernels_dont_have_tags = 0
how_many_kernels_have_tags = 0

for index, row in mt.iterrows():
    kernel_id = row["id"]
    tag_string = row["cvp_import"]
    if not isinstance(tag_string, str):
        how_many_kernels_dont_have_tags += 1
        tags_of_this_kernel = ["<(no_imports)>"]
    else:
        how_many_kernels_have_tags += 1
        tags_of_this_kernel = tag_string.split(";")
    for tag in tags_of_this_kernel:
        #####
        if tag in tags_kernel:
            tags_kernel[tag].append(kernel_id)
        else:
            tags_kernel[tag] = [kernel_id]
        #####
        if tag in tags_high_and_low_vote_kernel_number:
            if row["high_voted"]:
                tags_high_and_low_vote_kernel_number[tag][0] += 1
            else:
                tags_high_and_low_vote_kernel_number[tag][1] += 1
        else:
            tags_high_and_low_vote_kernel_number[tag] = [1, 0] if row["high_voted"] else [0, 1]
    kernel_tags[kernel_id] = tags_of_this_kernel

print(
    "how many kernels have imports: ", how_many_kernels_have_tags, "\n",
    "how_many_kernels_dont_have_imports: ", how_many_kernels_dont_have_tags, "\n",
    # "how many tags are in total: ", len(list(tags_num.keys())), "\n"
    tags_high_and_low_vote_kernel_number
)

def counting_tags(string):
    if isinstance(string, float):
        return 0
    return len(string.split(";"))

def target_encoding_tags_sum(string, return_stats="sum"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_imports)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "sum":
        return sum(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_avg(string, return_stats="avg"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_imports)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if (return_stats == "mean" or return_stats == "avg"):
        return mean(encoding_vals)
    else:
        return 0

def target_encoding_tags_max(string, return_stats="max"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_imports)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "max":
        return max(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_min(string, return_stats="min"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_imports)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "min":
        return min(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_median(string, return_stats="median"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_imports)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "median":
        return median(encoding_vals)
    else:
        return 0

mt["cvp_num_of_ipt"] = 0 ## import
mt["cvp_ipt_te_sum"] = None ## import, target encoding
mt["cvp_ipt_te_avg"] = None
mt["cvp_ipt_te_max"] = None
mt["cvp_ipt_te_min"] = None
mt["cvp_ipt_te_median"] = None

mt["cvp_num_of_ipt"] = mt["cvp_import"].map(counting_tags)
mt["cvp_ipt_te_sum"] = mt["cvp_import"].map(target_encoding_tags_sum)
mt["cvp_ipt_te_avg"] = mt["cvp_import"].map(target_encoding_tags_avg)
mt["cvp_ipt_te_max"] = mt["cvp_import"].map(target_encoding_tags_max)
mt["cvp_ipt_te_min"] = mt["cvp_import"].map(target_encoding_tags_min)
mt["cvp_ipt_te_median"] = mt["cvp_import"].map(target_encoding_tags_median)
# mt["km_target_encoding"] = 0
# mt["km_target_encoding"] = mt["km_tags"].map(target_encoding_tags)
mt.to_csv("kernel_datssource_code_1.csv", index=None)

how many kernels have imports:  27620 
 how_many_kernels_dont_have_imports:  585 
 {'logging': [94, 46], 'numpy': [13562, 13699], 'os': [4689, 3783], 'pandas': [13680, 14293], 'sys': [575, 402], 'xgboost': [1052, 1056], 'matplotlib.pyplot': [9340, 9006], 'seaborn': [6327, 5888], 'warnings': [2251, 1506], 'keras': [387, 323], 'keras.backend': [128, 66], 'tensorflow': [686, 624], 're': [1034, 904], 'gc': [1176, 468], 'matplotlib': [855, 748], 'datetime': [754, 705], 'lightgbm': [895, 355], 'time': [1123, 652], 'plotly.graph_objs': [1029, 618], 'plotly.offline': [520, 338], 'plotly.tools': [289, 192], 'ast': [74, 22], 'eli5': [31, 12], 'json': [380, 227], 'shap': [47, 9], 'matplotlib.image': [132, 133], 'missingno': [132, 71], 'sklearn': [273, 367], 'base64': [77, 25], 'codecs': [68, 36], 'nltk': [409, 331], 'io': [103, 47], 'itertools': [408, 319], 'squarify': [60, 18], 'folium': [120, 70], 'folium.plugins': [17, 5], 'matplotlib.patches': [126, 78], 'scipy': [255, 215], '<(no_imports)>':

### target encoding and some new features for import from

In [1]:
import pandas as pd
import os

# cvo: code_voted_original
# cvp: code_voted_pyscript
# km: kernel_measures
merged_table = r"J:\EECS6414\process\Pipeline\Data_cleaning\kernel_datssource_code_1.csv"
mt = pd.read_csv(merged_table, encoding="utf-8")

kernel_tags = {}
tags_kernel = {}
tags_high_and_low_vote_kernel_number = {}

how_many_kernels_dont_have_tags = 0
how_many_kernels_have_tags = 0

for index, row in mt.iterrows():
    kernel_id = row["id"]
    tag_string = row["cvp_import_from"]
    if not isinstance(tag_string, str):
        how_many_kernels_dont_have_tags += 1
        tags_of_this_kernel = ["<(no_import_from)>"]
    else:
        how_many_kernels_have_tags += 1
        tags_of_this_kernel = tag_string.split(";")
    for tag in tags_of_this_kernel:
        #####
        if tag in tags_kernel:
            tags_kernel[tag].append(kernel_id)
        else:
            tags_kernel[tag] = [kernel_id]
        #####
        if tag in tags_high_and_low_vote_kernel_number:
            if row["high_voted"]:
                tags_high_and_low_vote_kernel_number[tag][0] += 1
            else:
                tags_high_and_low_vote_kernel_number[tag][1] += 1
        else:
            tags_high_and_low_vote_kernel_number[tag] = [1, 0] if row["high_voted"] else [0, 1]
    kernel_tags[kernel_id] = tags_of_this_kernel

print(
    "how many kernels have import from: ", how_many_kernels_have_tags, "\n",
    "how_many_kernels_dont_have_import_from: ", how_many_kernels_dont_have_tags, "\n",
    # "how many tags are in total: ", len(list(tags_num.keys())), "\n"
    tags_high_and_low_vote_kernel_number
)

def counting_tags(string):
    if isinstance(string, float):
        return 0
    return len(string.split(";"))

def target_encoding_tags_sum(string, return_stats="sum"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_import_from)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "sum":
        return sum(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_avg(string, return_stats="avg"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_import_from)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if (return_stats == "mean" or return_stats == "avg"):
        return mean(encoding_vals)
    else:
        return 0

def target_encoding_tags_max(string, return_stats="max"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_import_from)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "max":
        return max(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_min(string, return_stats="min"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_import_from)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "min":
        return min(encoding_vals)
    else:
        return 0
    
def target_encoding_tags_median(string, return_stats="median"):
    import numpy as np
    from statistics import mean, median
    t = tags_high_and_low_vote_kernel_number
    if isinstance(string, float):
        tags = ["<(no_import_from)>"]
    else:
        tags = string.split(";")
    encoding_vals = []
    for tag in tags:
        encoding_vals.append(float(t[tag][0]) / sum(t[tag]))
    
    if return_stats == "median":
        return median(encoding_vals)
    else:
        return 0

mt["cvp_num_of_iptfrm"] = 0 ## import from
mt["cvp_iptfrm_te_sum"] = None ## import from, target encoding
mt["cvp_iptfrm_te_avg"] = None
mt["cvp_iptfrm_te_max"] = None
mt["cvp_iptfrm_te_min"] = None
mt["cvp_iptfrm_te_median"] = None

mt["cvp_num_of_iptfrm"] = mt["cvp_import_from"].map(counting_tags)
mt["cvp_iptfrm_te_sum"] = mt["cvp_import_from"].map(target_encoding_tags_sum)
mt["cvp_iptfrm_te_avg"] = mt["cvp_import_from"].map(target_encoding_tags_avg)
mt["cvp_iptfrm_te_max"] = mt["cvp_import_from"].map(target_encoding_tags_max)
mt["cvp_iptfrm_te_min"] = mt["cvp_import_from"].map(target_encoding_tags_min)
mt["cvp_iptfrm_te_median"] = mt["cvp_import_from"].map(target_encoding_tags_median)
# mt["km_target_encoding"] = 0
# mt["km_target_encoding"] = mt["km_tags"].map(target_encoding_tags)
mt.to_csv("kernel_datssource_code_2.csv", index=None)

how many kernels have import from:  22764 
 how_many_kernels_dont_have_import_from:  5441 
 {'keras.callbacks.EarlyStopping': [515, 276], 'keras.layers.Bidirectional': [188, 80], 'keras.layers.Conv1D': [191, 88], 'keras.layers.Flatten': [729, 597], 'keras.layers.GlobalMaxPooling1D': [120, 53], 'keras.layers.MaxPooling1D': [113, 68], 'keras.layers.SpatialDropout1D': [132, 48], 'keras.layers.core.Activation': [83, 112], 'keras.layers.core.Dense': [108, 143], 'keras.layers.core.Dropout': [99, 134], 'keras.layers.embeddings.Embedding': [29, 27], 'keras.layers.normalization.BatchNormalization': [128, 95], 'keras.layers.recurrent.GRU': [7, 5], 'keras.layers.recurrent.LSTM': [29, 18], 'keras.models.Sequential': [1082, 1024], 'keras.preprocessing.sequence': [126, 52], 'keras.preprocessing.text': [94, 29], 'keras.utils.np_utils': [187, 226], 'nltk.corpus.stopwords': [521, 364], 'nltk.word_tokenize': [61, 59], 'sklearn.decomposition': [46, 65], 'sklearn.decomposition.TruncatedSVD': [218, 152], '

### target encoding and other measures for km_isNotebook

In [1]:
import os
import pandas as pd

# cvo: code_voted_original
# cvp: code_voted_pyscript
# km: kernel_measures
merged_table = r"J:\EECS6414\process\Pipeline\Data_cleaning\kernel_datssource_code_2.csv"
mt = pd.read_csv(merged_table, encoding="utf-8")

tags_kernel = {}
tags_high_and_low_vote_kernel_number = {}

how_many_kernels_dont_have_tags = 0
how_many_kernels_have_tags = 0

for index, row in mt.iterrows():
    kernel_id = row["id"]
    tag = row["km_isNotebook"]
    if row["km_isNotebook"]:
        how_many_kernels_have_tags += 1
    else: 
        how_many_kernels_dont_have_tags += 1
    ##########
    if tag not in tags_kernel:
        tags_kernel[tag] = [kernel_id]
    else:
        tags_kernel[tag].append(kernel_id)
    ##########
    if tag in tags_high_and_low_vote_kernel_number:
        if row["high_voted"]:
            tags_high_and_low_vote_kernel_number[tag][0] += 1
        else:
            tags_high_and_low_vote_kernel_number[tag][1] += 1
    else:
        tags_high_and_low_vote_kernel_number[tag] = [1, 0] if row["high_voted"] else [0, 1]

print(
    "how many kernels are notebook: ", how_many_kernels_have_tags, "\n",
    "how_many_kernels_arent_notebook: ", how_many_kernels_dont_have_tags, "\n",
    # "how many tags are in total: ", len(list(tags_num.keys())), "\n"
    tags_high_and_low_vote_kernel_number
)

def target_encoding(tag):
    t = tags_high_and_low_vote_kernel_number
    return float(t[tag][0]) / sum(t[tag])

mt["km_isNotebook"] = mt["km_isNotebook"].map(target_encoding)
mt.to_csv("kernel_datssource_code_3.csv", index=None)

how many kernels are notebook:  23915 
 how_many_kernels_arent_notebook:  4290 
 {True: [11762, 12153], False: [2040, 2250]}


## average votes for tags/imports/importfroms

In [ ]:
def xxxx(item, mt):
    item_votes = {}
    for index, row in mt.iterrows():
        tag = row["km_votes"]